# Control and confounding in a single driver (Fig. 3a)

In [1]:
import sys
from pathlib import Path

import numpy as np
import plotly.graph_objects as go

from scipy.ndimage import gaussian_filter1d

from tigramite.causal_effects import CausalEffects

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 

In [ ]:
repo_root = Path().resolve().parent
sys.path.append(str(repo_root))

In [3]:
import functions.sliding_windows as sw
import functions.indicators_computation as ind
import functions.significance_testing as sgtest

In [4]:
def gaussian_smoothing(ts, sigma=30):
    trend = gaussian_filter1d(ts, sigma=sigma)    
    residuals = ts - trend
    return trend, residuals

In [5]:
def generate_time_series_RisX(t_len, sigma_Y, sigma_r, seed, model='base', r_model='r_slope_plus_ar1', ac_r=None, slope=None, gamma=1, len_ac_var=1, return_noise=True, bif_type='saddle-node'):
    random_state = np.random.default_rng(seed=seed)
    
    # Generate base random data
    dataR = random_state.normal(0, 0, size=(t_len, 1))
    dataY = random_state.normal(0, sigma_Y, size=(t_len, 1))
    data = np.hstack((dataR, dataY))

    dataEta = random_state.normal(0, sigma_r, size=(t_len, 1))

    # parameters
    h = 0.01  # time step
    initial_r = -1
    s=slope

    # Initial conditions
    data[0, 0] = initial_r
    if bif_type == 'pitchfork':
        data[0, 1] = 0
    if bif_type=='saddle-node':
        if r_model=='r_is_arma':
            data[0, 1] = 2.4
        else:
            data[0, 1] = 1.325


    if model == 'base':
        for t in range(1, t_len):
            data[t, 0] += h * s + data[t-1, 0]


    elif model == 'confounderDecreasAC' or model == 'confounderIncreasAC' or model == 'falseAlarm' or model=='r_is_arma_cst_ac':

        if model == 'confounderDecreasAC':
            beta_0 = 0.9
            beta_middle = 0.1
            k = (beta_middle - beta_0) / (t_len//len_ac_var)

        elif model == 'confounderIncreasAC' or model == 'falseAlarm':
            beta_0 = 0.1
            beta_final = 0.9
            k = (beta_final - beta_0) / (t_len//len_ac_var)
        for t in range(1, t_len):
            if model == 'confounderDecreasAC':
                beta_t = max(0, beta_0 + k * t)
            elif model == 'confounderIncreasAC' or model == 'falseAlarm':
                beta_t = min(beta_0 + k * t, beta_final)
            elif model == 'r_is_arma_cst_ac':
                beta_t = ac_r
                s=slope
            
            if r_model=='r_slope_plus_ar1':
                dataEta[t] += beta_t * dataEta[t-1]
                data[t, 0] += initial_r + h*s*t + h*dataEta[t]
            elif r_model=='r_is_arma':
                data[t, 0] += beta_t * data[t-1, 0] + initial_r + h*s*t + h*dataEta[t]

    for t in range(1, t_len):
        if bif_type == 'pitchfork':
            data[t, 1] += data[t-1, 1] + h * (gamma*data[t-1, 0] * data[t-1, 1] - data[t-1, 1]**3)
        
        if bif_type == 'saddle-node':
            data[t, 1] += data[t-1, 1] + h * (data[t-1, 1] - data[t-1, 1]**3 - gamma*data[t-1, 0])

    if not return_noise:
        return {'data': data}
    else:
        if model == 'base':
            noise = dataY.flatten()
        else:
            noise = dataY.flatten() + data[:, 0]
        return {'data': data, 'noise': noise}

In [6]:
t_len = 10000

In [7]:
window_length = 500
window_step = 1
time_windows, center_points = sw.get_centered_sliding_windows(window_length, window_step, t_len)

## Decreasing AC for r and bifurcation for y

In [8]:
seed=1

In [9]:
data_saddle_node_RisX = generate_time_series_RisX(t_len, sigma_Y=0.01, sigma_r=60, seed=seed, model='confounderDecreasAC', r_model='r_is_arma', ac_r=None, slope=0.015, gamma=1, len_ac_var=1.5, return_noise=True, bif_type='saddle-node')

/var/folders/lm/c2t_62cj4rg3ry50_0z2s5j00000gn/T/ipykernel_6374/2574992495.py:56: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data[t, 0] += beta_t * data[t-1, 0] + initial_r + h*s*t + h*dataEta[t]


In [10]:
dataY_saddle_node_RisX = data_saddle_node_RisX['data'][:, -1]

In [11]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=dataY_saddle_node_RisX, mode='lines', name='Increasing ac_r', line=dict(color='rgba(128, 128, 128, 1)')))
fig.update_layout(
    xaxis_title='Time',
    yaxis_title='State of 𝑌',
    width=500, height=400,
    xaxis=dict(tickvals=[0, 2000, 4000, 6000, 8000, 10000], ticktext=['0', '2,000', '4,000', '6,000', '8,000', '10,000'], range=[0, t_len]),
    legend=dict(x=0.03, y=0.05, traceorder='reversed', bgcolor='rgba(255, 255, 255, 0.5)'),
    plot_bgcolor='rgba(215, 238, 244, 0.3)',    # plotting area
    paper_bgcolor='rgba(0, 0, 0, 0)',  # entire plot background
)
fig.show()

In [12]:
# fig.write_image('fig_risX_stateOfTippingElement.svg')

In [13]:
dataR_saddle_node_RisX = data_saddle_node_RisX['data'][:, 0]

In [14]:
r_trend, r_residuals = gaussian_smoothing(data_saddle_node_RisX['data'][:, 0], sigma=60)

In [15]:
def plot_smoothing(ts, trend, residuals):
    fig = go.Figure()

    fig.add_trace(go.Scatter(y=ts, mode='lines', name='Original Time Series 𝑋̃', line=dict(color='#916F8A'), opacity=0.8))
    fig.add_trace(go.Scatter(y=residuals, mode='lines', name='Residuals 𝑋', line=dict(color='#DE8787'), opacity=0.8))
    fig.add_trace(go.Scatter(y=trend, mode='lines', name='Trend 𝑟 ', line=dict(color='#A02C5A')))

    fig.update_layout(
        xaxis_title='Time',
        yaxis_title='Value',
        xaxis=dict(tickvals=[0, 2000, 4000, 6000, 8000, 10000], ticktext=['0', '2,000', '4,000', '6,000', '8,000', '10,000'], range=[0, t_len]),
        legend=dict(x=0.43, y=0.05, traceorder='normal', bgcolor='rgba(255, 255, 255, 0.5)'),
        width=500, height=400,
        plot_bgcolor='rgba(215, 238, 244, 0.3)',    # plotting area
        paper_bgcolor='rgba(0, 0, 0, 0)',  # entire plot background
    )

    return fig

In [16]:
fig = plot_smoothing(dataR_saddle_node_RisX, r_trend, r_residuals)
fig.show()

In [17]:
# fig.write_image('fig_risX_value.svg')

In [18]:
data_saddle_node_RisX['data'][:, 0] = r_residuals

In [19]:
graph =  np.array([[['', '-->'], ['', '-->']], #r residuals
                   [['', ''], ['', '-->']], #y
                   ], dtype='<U3')

X = [(1,-1)]
Y = [(1,0)]

causal_effects = CausalEffects(graph, graph_type='stationary_dag', X=X, Y=Y, S=None, 
                               hidden_variables=None, verbosity=0)

In [20]:
bif_time = 9000

In [23]:
data_saddle_node_RisX_trunc = data_saddle_node_RisX['data'][:bif_time, :]

In [24]:
window_length = 300
window_step = 1
time_windows, center_points = sw.get_centered_sliding_windows(window_length, window_step, bif_time)

In [25]:
ac1_saddle_node_RisX = ind.regular_autocorrelation(dataY_saddle_node_RisX[:bif_time], time_windows, lag=1, detrend=True)

In [26]:
causalLinReg_saddle_node_RisX, _, _ = ind.pipeline_causalEE_weighted(
        causal_effects, data_saddle_node_RisX_trunc, LinearRegression(), StandardScaler(),
        n_points=150, time_windows=time_windows, var_names=[r'$r_residuals$', r'$Y$'],
        )

In [27]:
nb_surrogates = 10000

slope_AC1, intercept_AC1 = np.polyfit(center_points, ac1_saddle_node_RisX, 1)
pv_AC1, _ = sgtest.p_value_from_fourier(ac1_saddle_node_RisX, nb_surrogates, slope_AC1)

slope_DCE, intercept_DCE = np.polyfit(center_points, causalLinReg_saddle_node_RisX, 1)
pv_DCE, _ = sgtest.p_value_from_fourier(causalLinReg_saddle_node_RisX, nb_surrogates, slope_DCE)

In [28]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=center_points, y=ac1_saddle_node_RisX, mode='lines', name='AC1', line=dict(color='orange')))
if pv_AC1 < 0.001:
    fig.add_trace(go.Scatter(x=center_points, y=slope_AC1 * center_points + intercept_AC1, mode='lines', 
                             line=dict(color='orange', dash="dot"), name=f'p < 0.001'))
else:
    fig.add_trace(go.Scatter(x=center_points, y=slope_AC1 * center_points + intercept_AC1, mode='lines', 
                             line=dict(color='orange', dash="dot"), name=f'p = {pv_AC1: .3f}'))


fig.add_trace(go.Scatter(x=center_points+window_length/2, y=causalLinReg_saddle_node_RisX, mode='lines', name='DCE', line=dict(color='#00AAD4')))
if pv_DCE < 0.001:
    fig.add_trace(go.Scatter(x=center_points+window_length/2, y=slope_DCE * (center_points+window_length/2) + intercept_DCE, mode='lines', 
                             line=dict(color='#00AAD4', dash="dot"), name=f'p < 0.001'))
else:
    fig.add_trace(go.Scatter(x=center_points+window_length/2, y=slope_DCE * (center_points+window_length/2) + intercept_DCE,mode='lines', 
                             line=dict(color='#00AAD4', dash="dot"), name=f'p = {pv_DCE: .3f}'))

fig.update_layout(xaxis_title='Time', 
                  yaxis_title='CSD indicator',
                  width=500, height=400,
                  legend=dict(x=0.64, y=0.04, traceorder='normal', bgcolor='rgba(255, 255, 255, 0.5)'),
                  xaxis=dict(range=[0, 10000], tickvals=[0, 2000, 4000, 6000, 8000, 10000], ticktext=['0', '2,000', '4,000', '6,000', '8,000', '10,000']),
                  yaxis=dict(range=[0.87, 1.01], tickvals=[0.9, 0.95, 1], ticktext=['0.9', '0.95', '1']),
                    plot_bgcolor='rgba(215, 238, 244, 0.3)',    # plotting area
                    paper_bgcolor='rgba(0, 0, 0, 0)',  # entire plot background

)
fig.show()

In [ ]:
# fig.write_image('fig_risX_indicators.svg')